In [7]:
pip --version

python -m pip install --upgrade pip

pip3 install tensorflow==2.0.0

SyntaxError: invalid syntax (3987725962.py, line 3)

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Load the dataset
raw_data = pd.read_csv('NGSIM__Lankershim_Vehicle_Trajectories.csv')
raw_df = pd.DataFrame(raw_data)

# select intersection: 1234(5?)
# Direction: 1E2N3W4S
# Movement: 1TH, 2LT, 3RT

intID = 2
Movement = 1
Direction = 1

df = raw_df.loc[(raw_df['Int_ID'] == intID)]
df = df.loc[df['Movement'] == Movement]
nV = len(set(df.Vehicle_ID))
print('The total volume number is ' + str(nV))   

# Time stamp process

# sort and pre-normalize the global time
df.Global_Time = (df.Global_Time - min(df['Global_Time']))/1000 # changed unit to seconds # (ten frames per second)
# nT = len(df.Global_Time)

# plot the Global_Time
# number = np.arange(0,len(df))
# mp.plot(number, df.Global_Time)
# print(max(df.Global_Time)) # time spans

# set the timestamp
time_df = df[['Vehicle_ID', 'Global_Time']] # Select only the two columns from complete dataframe
stp_df = pd.DataFrame(columns=['TimeStamp','Volume'])
for i in range(math.ceil(max(df.Global_Time)/10)):
    temp_df = time_df.loc[(df['Global_Time'] >= i * 10) & (df['Global_Time'] <= (i+1) * 10)]
    temp_n = len(set(temp_df.Vehicle_ID))
    data={'TimeStamp':[i],'Volume':[temp_n]}
    new_row = pd.DataFrame(data)
    stp_df = pd.concat([stp_df,new_row],ignore_index=True)    
    i = i + 1
else:
    print("Finish loop")
# Cut the last time stamp for lacking sufficient time
stp_df = stp_df.iloc[:-1 , :]
stp_df # display the whole time stamps

In [ ]:

# Convert the 'date_time' column to datetime format and set it as the index
df['date_time'] = pd.to_datetime(df['date_time'])
df = df.set_index('date_time')

# Extract the traffic volume data
traffic_volume = df['traffic_volume'].values

# Normalize the traffic volume data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
traffic_volume = scaler.fit_transform(traffic_volume.reshape(-1, 1))

# Split the data into training and testing sets
train_size = int(len(traffic_volume) * 0.8)
test_size = len(traffic_volume) - train_size
train_data, test_data = traffic_volume[0:train_size,:], traffic_volume[train_size:len(traffic_volume),:]
